# snowflake_connector

In [ ]:
#| default_exp snowflake_connector

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.basics import patch_to
from fastcore.test import *

from typing import List, Tuple, Dict, Any, Callable, Optional

import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

import pandas as pd
import json, re, os
from datetime import datetime
import pytz

# SnowflakeConnector

Connector class for Snowflake operations related to Slack interactions. Handles connecting to Snowflake and provides methods for data operations.

In [ ]:
#| export

class SnowflakeConnector:
    """
    Connector class for Snowflake operations related to Slack interactions.
    Handles connecting to Snowflake and provides methods for data operations.
    """
    
    def __init__(self, connection_params: Optional[Dict[str, Any]] = None):
        """Initialize the Snowflake connector.
        
        Args:
            connection_params: Dictionary with connection parameters.
                If None, will use environment variables.
        """
        # Use provided params or get from environment
        if connection_params:
            self.connection_params = connection_params
        else:
            self.connection_params = {
                'account': os.environ.get('SNOWFLAKE_ACCOUNT'),
                'user': os.environ.get('SNOWFLAKE_USER'),
                'password': os.environ.get('SNOWFLAKE_PASSWORD'),
                'warehouse': os.environ.get('SNOWFLAKE_WAREHOUSE'),
                'database': os.environ.get('SNOWFLAKE_DATABASE'),
                'schema': os.environ.get('SNOWFLAKE_SCHEMA')
            }
        
        # Connection is initialized lazily when needed
        self._conn = None
        self._schema_cache = {}
        self.database = self.connection_params['database']
        self.schema = self.connection_params['schema']  

First, if we want to make a Snowfalke Connection class, we should probably make a way to connect to Snowflake...

In [ ]:
#| export

@patch_to(SnowflakeConnector, cls_method=True)
def _get_connection(self):
    """Get a Snowflake connection, creating one if needed."""
    if not hasattr(self, '_conn'):
        self._conn = snowflake.connector.connect(
            **self.connection_params
        )
    return self._conn

Yay! We are now able to connect to Snowflake which is pretty cool. Let's also make a method to close the connection when we are done with it. This is important because we don't want to leave connections open and use up resources. We can do this by adding a `close` method to our class. This method will call the `close` method on the connection object. We can also add a check to see if the connection is already closed before trying to close it again.

In [ ]:
#| export
@patch_to(SnowflakeConnector, cls_method=True)
def close(self):
        """Close the Snowflake connection if open."""
        if self._conn:
            self._conn.close()
            self._conn = None

Awesome, now we can connect to Snowflake and close the connection when we are done with it. Let's also add a method to execute a query and return the results. This will be useful for getting data from Snowflake. We can do this by adding an `execute_query` method to our class. This method will take a query string as an argument and return the results of the query. We can use the `cursor` object to execute the query and fetch the results.

In [ ]:
#| export
@patch_to(SnowflakeConnector, cls_method=True)
def execute_query(self, query: str, params: Optional[List[Any]] = None) -> List[Dict[str, Any]]:
    """Execute a query and return results as a list of dictionaries.
    
    Args:
        query: SQL query to execute
        params: Optional parameters for the query
        
    Returns:
        List of dictionaries with query results
    """
    try:
        conn = self._get_connection()
        cursor = conn.cursor()
        
        # Execute the query
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        
        # Get column names
        columns = [desc[0] for desc in cursor.description]
        
        # Convert results to list of dictionaries
        results = []
        for row in cursor:
            results.append(dict(zip(columns, row)))
        
        return results
        
    except Exception as e:
        print(f"Error executing query: {e}")
        raise

## Data Insertion

Let's also create a way to insert data into Snowflake.

In [ ]:
#| export

@patch_to(SnowflakeConnector, cls_method=True)
def _get_table_schema(self, table_name: str, use_cache: bool = True) -> Dict[str, str]:
    """
    Query Snowflake to get the schema of a table.
    
    Args:
        table_name: Name of the table
        use_cache: Whether to use cached schema information
        
    Returns:
        Dictionary mapping column names to their data types
    """
    # Use instance values as defaults if not provided
    database = self.database
    schema = self.schema
    
    if not database or not schema:
        raise ValueError("Database and schema must be provided")
    
    # Check cache first
    cache_key = f"{database}.{schema}.{table_name}"
    if use_cache and cache_key in self._schema_cache:
        return self._schema_cache[cache_key]
        
    schema_query = f"""
        SELECT 
            COLUMN_NAME, 
            DATA_TYPE 
        FROM 
            {database}.INFORMATION_SCHEMA.COLUMNS 
        WHERE 
            TABLE_SCHEMA = '{schema}' 
            AND TABLE_NAME = '{table_name}'
    """
    
    try:
        conn = self._get_connection()
        with conn.cursor() as cs:
            cs.execute(schema_query)
            results = cs.fetchall()
            result_schema = {row[0]: row[1] for row in results}
            
            # Cache the result
            if use_cache:
                self._schema_cache[cache_key] = result_schema
                
            return result_schema
    except Exception as e:
        print(f"Error getting schema: {e}")
        raise RuntimeError(f"Failed to get schema for {table_name}: {str(e)}")


@patch_to(SnowflakeConnector, cls_method=True)
def _get_current_timestamp(self, timezone: str = 'America/Chicago') -> str:
    """Get current timestamp in the specified timezone."""
    tz = pytz.timezone(timezone)
    return datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")

@patch_to(SnowflakeConnector, cls_method=True)
def _create_schema_mapping(self, db_schema: Dict) -> Dict[str, str]:
    """Create a case-insensitive mapping of schema columns."""
    return {k.upper(): k for k in db_schema.keys()}


@patch_to(SnowflakeConnector, cls_method=True)
def _prepare_insert_data(
        self,
        insert_data: Dict, 
        db_schema: Dict,
        schema_keys_map: Dict[str, str],
        auto_timestamp: bool = True,
        include_all_columns: bool = False,
        timezone: str = 'America/Chicago',
        debug: bool = False
    ) -> Dict:
    """
    Prepare and filter data for insertion based on schema.
    """
    filtered_data = {}
    current_time = self._get_current_timestamp(timezone)
    
    if include_all_columns:
        for schema_col in db_schema.keys():
            filtered_data[schema_col] = None
    
    for k, v in insert_data.items():
        k_upper = k.upper()
        if k_upper in schema_keys_map:
            # Use the actual case from the schema
            original_case_key = schema_keys_map[k_upper]
            filtered_data[original_case_key] = v
    
    if auto_timestamp:
        timestamp_fields = {
            'UPDATED_AT': 'updated_at',
            'CREATED_AT': 'created_at',
            'RUN_TIME': 'run_time',
            'INSERTED_AT': 'inserted_at',
            'TIMESTAMP': 'timestamp'
        }
        
        for upper_field, lower_field in timestamp_fields.items():
            if upper_field in schema_keys_map:
                original_case = schema_keys_map[upper_field]
                
                field_provided = False
                for k in insert_data.keys():
                    if k.upper() == upper_field:
                        field_provided = True
                        break
                
                if not field_provided:
                    filtered_data[original_case] = current_time
    
    if debug and not filtered_data:
        print(f"Available columns in schema: {list(db_schema.keys())}")
        print(f"Provided keys: {list(insert_data.keys())}")
        print(f"Case-insensitive mapping: {schema_keys_map}")
        
    return filtered_data


@patch_to(SnowflakeConnector, cls_method=True)
def _build_insert_query(self, table_name: str, columns: List[str]) -> str:
    """Build the base INSERT query for Snowflake."""
    return f"""
        INSERT INTO {self.database}.{self.schema}.{table_name} (
            {', '.join(columns)}
        )
        SELECT
    """

@patch_to(SnowflakeConnector, cls_method=True)
def _prepare_values_and_expressions(
    self,
    filtered_data: Dict, 
    columns: List[str], 
    db_schema: Dict
    ) -> Tuple[List, List[str]]:
    """Prepare values and SQL expressions for the query."""
    values = []
    select_exprs = []
    
    for col in columns:
        value = filtered_data[col]
        data_type = db_schema[col].upper()
        
        # Handle nulls consistently
        if value is None:
            select_exprs.append("NULL")
            continue
            
        if any(array_type in data_type for array_type in ['ARRAY', 'LIST']):
            if isinstance(value, list) and len(value) == 0:
                select_exprs.append("ARRAY_CONSTRUCT()")
            elif isinstance(value, list):
                placeholders = ", ".join(["%s"] * len(value))
                select_exprs.append(f"ARRAY_CONSTRUCT({placeholders})")
                values.extend(value)
            else:
                # Handle single value as array
                select_exprs.append("ARRAY_CONSTRUCT(%s)")
                values.append(value)
        
        elif any(obj_type in data_type for obj_type in ['OBJECT', 'VARIANT', 'JSON']):
            # Handle JSON object with PARSE_JSON - convert to string first
            select_exprs.append("PARSE_JSON(%s)")
            # Always convert to JSON string, even if it's already a dict
            if isinstance(value, (dict, list)):
                values.append(json.dumps(value))
            elif isinstance(value, str):
                # If it's already a string, validate it's valid JSON
                try:
                    json.loads(value)  # Validate JSON
                    values.append(value)
                except json.JSONDecodeError:
                    # If not valid JSON, wrap it as a string value
                    values.append(json.dumps(value))
            else:
                # For other types, convert to JSON
                values.append(json.dumps(value))
        
        else:
            select_exprs.append("%s")
            values.append(value)
    
    return values, select_exprs

# Additional helper method to validate JSON data before insertion
@patch_to(SnowflakeConnector, cls_method=True)
def _validate_json_data(self, data: Dict[str, Any], db_schema: Dict) -> Dict[str, Any]:
    """
    Validate and convert JSON data types before insertion.
    
    Args:
        data: Data dictionary to validate
        db_schema: Table schema dictionary
        
    Returns:
        Validated data dictionary with properly formatted JSON strings
    """
    validated_data = {}
    
    for key, value in data.items():
        if key in db_schema:
            data_type = db_schema[key].upper()
            
            # Handle JSON/VARIANT/OBJECT types
            if any(obj_type in data_type for obj_type in ['OBJECT', 'VARIANT', 'JSON']):
                if value is None:
                    validated_data[key] = None
                elif isinstance(value, str):
                    # Validate existing JSON string
                    try:
                        json.loads(value)
                        validated_data[key] = value
                    except json.JSONDecodeError:
                        # Convert invalid JSON string to valid JSON
                        validated_data[key] = json.dumps(value)
                else:
                    # Convert dict, list, or other types to JSON string
                    validated_data[key] = json.dumps(value)
            else:
                validated_data[key] = value
        else:
            validated_data[key] = value
    
    return validated_data

@patch_to(SnowflakeConnector, cls_method=True)
def _execute_insert_query(
    self,
    database: str,
    schema: str,
    query: str,
    values: List,
    debug: bool = False
    ) -> bool:
    """Execute the INSERT query on Snowflake."""
    try:
        if debug:
            print("DEBUG SQL:", query)
            print("DEBUG values:", values)
            print("DEBUG values count:", len(values))
            
        conn = self._get_connection()
        with conn.cursor() as cs:
            cs.execute(query, values)
            conn.commit()
        
        return True
    
    except Exception as e:
        print(f"Error inserting data: {e}")
        if debug:
            print(f"Failed SQL: {query}")
            print(f"Values: {values}")
        raise RuntimeError(f"Failed to insert data: {str(e)}")


@patch_to(SnowflakeConnector, cls_method=True)
def insert_record(self, table_name: str, data: Dict[str, Any], **kwargs) -> Any:
    """
    Enhanced function to insert data into Snowflake with improved type handling.
    
    Args:
        table_name: Name of the table to insert into
        data: Dictionary of data to insert
        **kwargs: Additional options for insertion
        
    Returns:
        ID of the inserted record (if id_field is provided) or True if successful
    """
    # Set default options
    options = {
        'database': self.database,
        'schema': self.schema,
        'auto_timestamp': True,
        'timezone': 'America/Chicago',
        'debug': False,
        'include_all_columns': False
    }
    
    # Update with any provided options
    options.update(kwargs)
    
    if not options['database'] or not options['schema']:
        raise ValueError("Database and schema must be provided")
    
    # Get table schema
    db_schema = self._get_table_schema(table_name)
    print('db_schema', db_schema, 'sc')
    if not db_schema:
        raise ValueError(f"Could not retrieve schema for {options['database']}.{options['schema']}.{table_name}")
        
    if options['debug']:
        print(f"Table schema: {list(db_schema.keys())}")
        
    # Create case-insensitive mapping
    schema_keys_map = self._create_schema_mapping(db_schema)
    
    # Prepare and filter data
    filtered_data = self._prepare_insert_data(
        data,
        db_schema,
        schema_keys_map,
        options['auto_timestamp'],
        options['include_all_columns'],
        options['timezone'],
        options['debug']
    )
    
    # Nothing to insert
    if not filtered_data:
        print("Warning: No valid columns provided for insert")
        return False
    
    # Build the query
    columns = list(filtered_data.keys())
    insert_query = self._build_insert_query(table_name,columns)
    
    # Prepare values and expressions
    values, select_exprs = self._prepare_values_and_expressions(
        filtered_data, 
        columns, 
        db_schema
    )
    
    # Complete the query
    insert_query += ", ".join(select_exprs)
    
    self._execute_insert_query(
        options['database'],
        options['schema'],
        insert_query,
        values,
        options['debug']
    )
    
    return True

Now that we can insert individual rows, let's also add a method to insert multiple rows at once

In [ ]:
#| export

@patch_to(SnowflakeConnector, cls_method=True)
def bulk_insert(self, table_name: str, df: pd.DataFrame, **kwargs) -> bool:
    """
    Enhanced function to bulk insert DataFrame data into Snowflake with improved type handling.
    
    Args:
        table_name: Name of the table to insert into
        df: DataFrame with columns matching table structure
        **kwargs: Additional options for insertion
            - database: Override default database
            - schema: Override default schema
            - auto_timestamp: Add timestamps to standard timestamp fields (default: True)
            - timezone: Timezone for timestamps (default: 'America/Chicago')
            - debug: Enable debug output (default: False)
            - batch_size: Rows per batch (default: 10000)
            - quote_identifiers: Whether to quote identifiers (default: False)
            - chunk_size: Size of DataFrame chunks to process (default: None)
            - auto_create_table: Create table if not exists (default: False)
            - use_column_mapping: Map DataFrame columns to table columns (default: True)
        
    Returns:
        True if successful
    """
    # Set default options
    options = {
        'database': self.database,
        'schema': self.schema,
        'auto_timestamp': True,
        'timezone': 'America/Chicago',
        'debug': False,
        'batch_size': 10000,
        'quote_identifiers': False,
        'chunk_size': None,
        'auto_create_table': False,
        'use_column_mapping': True
    }
    
    # Update with any provided options
    options.update(kwargs)
    
    if not options['database'] or not options['schema']:
        raise ValueError("Database and schema must be provided")

    # Get table schema
    try:
        db_schema = self._get_table_schema(table_name)
    except Exception as e:
        if options['debug']:
            print(f"Error getting schema: {e}")
        if options['auto_create_table']:
            # Auto-create table logic would go here
            # Not implemented in this version
            raise NotImplementedError("Auto-create table not implemented yet")
        else:
            raise ValueError(f"Could not retrieve schema for {options['database']}.{options['schema']}.{table_name}")

    if options['debug']:
        print(f"Table schema: {list(db_schema.keys())}")
        print(f"DataFrame columns: {list(df.columns)}")

    # Create case-insensitive mapping
    schema_keys_map = self._create_schema_mapping(db_schema)
    
    # Process DataFrame to match schema if needed
    if options['use_column_mapping']:
        df_processed = self._prepare_dataframe(
            df,
            db_schema,
            schema_keys_map,
            options['auto_timestamp'],
            options['timezone'],
            options['debug']
        )
    else:
        df_processed = df.copy()
    
    if df_processed.empty:
        print("Warning: No valid data to insert after processing")
        return False
    
    try:
        conn = self._get_connection()
        
        # Set database and schema context
        qualified_table = f"{options['database']}.{options['schema']}.{table_name}"
        
        if options['debug']:
            print(f"Inserting into: {qualified_table}")
            print(f"Processed DataFrame shape: {df_processed.shape}")
        
        # Use the Snowflake Pandas integration
        success, num_chunks, num_rows, output = write_pandas(
            conn=conn,
            df=df_processed,
            table_name=qualified_table,
            quote_identifiers=options['quote_identifiers'],
            chunk_size=options['chunk_size'],
            compression='gzip',  # Usually a good default
            parallel=4,          # Use parallel processing
            overwrite=False,     # Append mode
            auto_create_table=False  # We handle schema validation separately
        )
        
        if success:
            #print(f"Bulk insert into {qualified_table}: {num_rows} rows in {num_chunks} chunks")
            return True
        else:
            print(f"Bulk insert failed: {output}")
            return False
            
    except Exception as e:
        if options['debug']:
            print(f"Error bulk inserting records: {e}")
            print(f"DataFrame columns: {list(df_processed.columns)}")
            print(f"First row: {df_processed.iloc[0].to_dict() if not df_processed.empty else 'Empty DataFrame'}")
            
        raise RuntimeError(f"Failed to bulk insert data: {str(e)}")


@patch_to(SnowflakeConnector, cls_method=True)
def _prepare_dataframe(
        self,
        df: pd.DataFrame,
        db_schema: Dict,
        schema_keys_map: Dict[str, str],
        auto_timestamp: bool = True,
        timezone: str = 'America/Chicago',
        debug: bool = False
    ) -> pd.DataFrame:
    """
    Prepare DataFrame for insertion based on table schema.
    
    Args:
        df: Input DataFrame
        db_schema: Table schema dictionary
        schema_keys_map: Case-insensitive column mapping
        auto_timestamp: Whether to add timestamps automatically
        timezone: Timezone for timestamps
        debug: Enable debug output
        
    Returns:
        Processed DataFrame ready for insertion
    """
    # Create a new DataFrame with correctly cased columns
    result_df = pd.DataFrame()
    current_time = self._get_current_timestamp(timezone)
    
    # Add all schema columns that exist in input DataFrame
    for col in db_schema.keys():
        col_upper = col.upper()
        
        # Check if column exists in input DataFrame (case-insensitive)
        matching_cols = [c for c in df.columns if c.upper() == col_upper]
        
        if matching_cols:
            # Use the first matching column from input
            input_col = matching_cols[0]
            result_df[col] = df[input_col].copy()
            
            # Handle special types based on schema
            data_type = db_schema[col].upper()
            
            # Convert array types
            if any(array_type in data_type for array_type in ['ARRAY', 'LIST']):
                # Ensure values are properly formatted as lists
                result_df[col] = result_df[col].apply(
                    lambda x: x if (isinstance(x, list) or x is None) else [x]
                )
                
            # Convert object/JSON types - FIX: Convert to JSON strings properly
            elif any(obj_type in data_type for obj_type in ['OBJECT', 'VARIANT', 'JSON']):
                # Convert values to JSON strings for DataFrame bulk insert
                def convert_to_json_string(x):
                    if x is None:
                        return None
                    elif isinstance(x, str):
                        # If it's already a string, validate and return as-is
                        try:
                            json.loads(x)  # Validate JSON
                            return x
                        except json.JSONDecodeError:
                            # If not valid JSON, wrap it as a JSON string
                            return json.dumps(x)
                    else:
                        # Convert dict, list, or other types to JSON string
                        return json.dumps(x)
                
                result_df[col] = result_df[col].apply(convert_to_json_string)
    
    # Add timestamp columns if requested
    if auto_timestamp:
        timestamp_fields = {
            'UPDATED_AT': 'updated_at',
            'CREATED_AT': 'created_at',
            'RUN_TIME': 'run_time',
            'INSERTED_AT': 'inserted_at',
            'TIMESTAMP': 'timestamp'
        }
        
        for upper_field, lower_field in timestamp_fields.items():
            if upper_field in schema_keys_map:
                original_case = schema_keys_map[upper_field]
                
                # Skip if column already exists in result DataFrame
                if original_case in result_df.columns:
                    continue
                    
                # Add timestamp column
                result_df[original_case] = current_time
    
    if debug:
        print(f"Original columns: {df.columns.tolist()}")
        print(f"Processed columns: {result_df.columns.tolist()}")
        print(f"Schema columns: {list(db_schema.keys())}")
        
    return result_df

In [ ]:
#| export

@patch_to(SnowflakeConnector, cls_method=True)    
def get_user_interactions(self, user_id: str, limit: int = 100) -> List[Dict[str, Any]]:
    """Get recent interactions for a specific user.
    
    Args:
        user_id: Slack user ID
        limit: Maximum number of records to return
        
    Returns:
        List of interaction records
    """
    query = f"""
        SELECT * 
        FROM SLACK_INTERACTIONS 
        WHERE USER_ID = %s 
        ORDER BY TIMESTAMP DESC 
        LIMIT {limit}
    """
    return self.execute_query(query, [user_id])

@patch_to(SnowflakeConnector, cls_method=True)
def get_interactions_by_view(self, view: str, limit: int = 100) -> List[Dict[str, Any]]:
    """Get recent interactions for a specific view.
    
    Args:
        view: View name
        limit: Maximum number of records to return
        
    Returns:
        List of interaction records
    """
    query = f"""
        SELECT * 
        FROM SLACK_INTERACTIONS 
        WHERE VIEW = %s 
        ORDER BY TIMESTAMP DESC 
        LIMIT {limit}
    """
    return self.execute_query(query, [view])

@patch_to(SnowflakeConnector, cls_method=True)
def get_interaction_summary(self) -> List[Dict[str, Any]]:
    """Get a summary of interactions by view and action type.
    
    Returns:
        Summary statistics
    """
    query = """
        SELECT 
            VIEW,
            ACTION_TYPE,
            COUNT(*) AS INTERACTION_COUNT,
            COUNT(DISTINCT USER_ID) AS UNIQUE_USERS,
            MIN(TIMESTAMP) AS FIRST_INTERACTION,
            MAX(TIMESTAMP) AS LATEST_INTERACTION
        FROM SLACK_INTERACTIONS
        GROUP BY VIEW, ACTION_TYPE
        ORDER BY VIEW, INTERACTION_COUNT DESC
    """
    return self.execute_query(query)

def __del__(self):
    """Ensure connection is closed when object is destroyed."""
    self.close()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()